In [20]:
import re                                                              
import os                                                              
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt                                        
import matplotlib.animation as animation
plt.style.use("ggplot")

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from tqdm import tqdm                                                 
tqdm.pandas()
import scipy.stats as stats

import gensim
from gensim import models
from gensim import corpora
from gensim.utils import simple_preprocess
from collections import defaultdict

import nltk
from nltk.stem import SnowballStemmer   
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
#nltk.download('stopwords')
#nltk.download('omw-1.4')
#nltk.download('wordnet')

import spacy
from spacy.lang.pt.examples import sentences 

from unidecode import unidecode

import string

In [21]:
path = os.getcwd()

df = pd.read_csv(path + "/tweets_brasil_1.csv")

/Users/giovannachaves/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (29,30,32,40,42,43,44,50,68) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [22]:
tweets = df[['id','text','created_at','author_id','author.name','author.description',
             'author.entities.description.hashtags','in_reply_to_username','quoted_username','geo.full_name',
             'geo.coordinates.coordinates','Brasilia','SaoPaulo','RiodeJaneiro','Fortaleza','Salvador']].copy()

In [23]:
tweets.head(20)

,id,text,created_at,author_id,author.name,author.description,author.entities.description.hashtags,in_reply_to_username,quoted_username,geo.full_name,geo.coordinates.coordinates,Brasilia,SaoPaulo,RiodeJaneiro,Fortaleza,Salvador
0,1614412011743461377,@TyraMain @GShinobiz SIM MANO! é minha kunoich...,2023-01-14T23:59:52.000Z,3342006537,hela,born to flop,NaN,TyraMain,NaN,"Brasilia, Brasil",NaN,1,0,0,0,0
1,1614411905023500289,@delucca Terrorista &amp; criminosa,2023-01-14T23:59:27.000Z,1435651588773883912,PeelLineBsB,Distribuidor OFICIAL PeelLine Brasília-DF\nDer...,NaN,delucca,NaN,"Brasilia, Brasil",NaN,1,0,0,0,0
2,1614411869120335873,@Marc31ox @dricacrf @C4RIM https://t.co/hBJAWf...,2023-01-14T23:59:18.000Z,210620557,Marcos Martinho,"Conflitos psicológicos atenuados, temática com...",NaN,Marc31ox,NaN,"Brasilia, Brasil",NaN,1,0,0,0,0
3,1614411742632898561,"Sábado quem vem já tem Anitta, tenho nem roupa...",2023-01-14T23:58:48.000Z,1175621317653082112,RooDrigo~⚡,"É que meu estilo é caro, sedutor e perigoso",NaN,NaN,NaN,"Brasilia, Brasil",NaN,1,0,0,0,0
4,1614411722005118976,Mentira https://t.co/svEjdEHCzU,2023-01-14T23:58:43.000Z,756533922855522304,𝔸𝕗𝕣𝕠𝕕𝕚𝕥𝕖,https://t.co/mMfrG36Pkk,NaN,NaN,choquei,"Brasilia, Brasil",NaN,1,0,0,0,0
5,1614411639465320461,hahahahahahhaha https://t.co/nRcAz2Lul4,2023-01-14T23:58:23.000Z,151696294,lucas sabino,designer e whovian de carteirinha.,NaN,NaN,oQuarterback,"Brasilia, Brasil",NaN,1,0,0,0,0
6,1614411626299494406,eu amo como minha família é sempre receptiva\n...,2023-01-14T23:58:20.000Z,555600264,Dani ✨,Juro que já tava assim quando cheguei,NaN,NaN,NaN,"Brasilia, Brasil",NaN,1,0,0,0,0
7,1614411605579554820,Mas todo mundo queria receber https://t.co/7g3...,2023-01-14T23:58:15.000Z,1112949432,ℌ𝔦𝔯𝔬𝔰𝔥𝔦𝔪𝔞ᶜʳᶠ,@amandeinudes 🖤// @Flamengo❤🖤,NaN,NaN,vanglockkk,"Brasilia, Brasil",NaN,1,0,0,0,0
8,1614411565578555393,@eduardobarao Mas essa família tradicional bra...,2023-01-14T23:58:06.000Z,254649333,Ana,NaN,NaN,eduardobarao,NaN,"Brasilia, Brasil",NaN,1,0,0,0,0
9,1614411390357504002,Quem n tem TAG nunca vai entender como é convi...,2023-01-14T23:57:24.000Z,1116937083677966338,ba,this is a fan account 🙃,NaN,NaN,NaN,"Brasilia, Brasil",NaN,1,0,0,0,0


In [24]:
text_col = tweets['text'].to_list()

text_col

['@TyraMain @GShinobiz SIM MANO! é minha kunoichi fav mas já tá na hora de dar espaço pra outros... tem tanta respostas que o kishimoto deve e o povo desperdiça as chance de ter',
 '@delucca Terrorista &amp; criminosa',
 '@Marc31ox @dricacrf @C4RIM https://t.co/hBJAWfm4E4',
 'Sábado quem vem já tem Anitta, tenho nem roupa ainda',
 'Mentira https://t.co/svEjdEHCzU',
 'hahahahahahhaha https://t.co/nRcAz2Lul4',
 'eu amo como minha família é sempre receptiva\\n\\no Uber veio nos deixar em casa, ofereceram água, café e ainda deram um bolinho de goiaba pro moço',
 'Mas todo mundo queria receber https://t.co/7g3fgZSZGO',
 '@eduardobarao Mas essa família tradicional brasileira soberba,  violenta e preconceituosa ainda existe.  Dia 8 de janeiro, ela se mostrou em toda sua virulência.',
 'Quem n tem TAG nunca vai entender como é conviver com isso kkkkkk.',
 '@euovitinn ** ***** ** ** ****** *****',
 'Fico feliz com as pessoas percebendo isso e minha cabeça fica: ai ta veno vc conseguiu',
 '@quim

In [25]:
sw_nltk = stopwords.words('portuguese')

new_stopwords = ['tipo', 'vei', 'mano', 'ei', 'to', 'dps', 'eh', 'tá', 'tão', 'tava', 'tavam', 'cê', 'mt', 'mto', 
                 'n', 'tbm', 'tb', 'qqlr', 'qualquer', 'gnt', 'gente', 'cara']

sw_nltk.extend(new_stopwords)

# removing accents from the stopwords
sw_nltk = [unidecode(item, errors="preserve") for item in sw_nltk]

In [26]:
def preprocess(text):
    tweets = []
    for tweet in text:
        text_lower = tweet.lower() # lowercase
        text_num = re.sub(r'\d+', '', text_lower) # remove numbers
        text_line = re.sub('\n', ' ', text_num) # transforming line break into whitespace
        text_double = re.sub(' +', ' ', text_line) # remove double spaces
        text_and = re.sub('amp', 'e', text_double) # replace & for e
        text_mention = re.sub('@[^\s]*', "", text_and) # remove @ and everything that comes after
        text_punct = re.sub('[%s]' % re.escape(string.punctuation), '' , text_mention) # remove punctuation
        text_acc = unidecode(text_punct, errors="preserve") # remove accents
        text_links = re.sub('http[^\s]*', "", text_acc)# remove links
        text_stop = " ".join([word for word in text_links.split() if word not in sw_nltk]) # remove stopwords
        tweets.append(text_stop)
    return tweets

In [34]:
tweets_list = preprocess(text_col)

Descriptive stats: number of unique IDs per day before and after the invasion, number of total tweets

In [36]:
nlp = spacy.load("pt_core_news_sm")
for tweet in tweets_list:
    doc = nlp(tweet)
    for token in doc:
        print(token.text, token.pos_)

sim ADV
kunoichi VERB
fav NOUN
hora NOUN
dar VERB
espaco ADJ
pra ADP
outros DET
tanta DET
respostas NOUN
kishimoto ADJ
deve VERB
povo NOUN
desperdica ADJ
chance VERB
ter VERB
terrorista NOUN
criminosa ADJ
sabado VERB
vem VERB
anitta PROPN
roupa NOUN
ainda ADV
mentira NOUN
hahahahahahhaha NOUN
amo NOUN
familia VERB
sempre ADV
receptivanno NOUN
uber DET
veio VERB
deixar VERB
casa NOUN
ofereceram VERB
agua ADJ
cafe VERB
ainda ADV
deram VERB
bolinho NOUN
goiaba VERB
pro ADV
moco ADV
todo DET
mundo PRON
queria VERB
receber VERB
familia VERB
tradicional ADJ
brasileira ADJ
soberba NOUN
violenta ADJ
preconceituosa ADJ
ainda ADV
existe VERB
dia NOUN
janeiro NOUN
mostrou VERB
toda DET
virulencia NOUN
tag ADV
nunca ADV
vai AUX
entender VERB
conviver VERB
kkkkkk PROPN
fico ADJ
feliz ADJ
pessoas NOUN
percebendo VERB
cabeca NOUN
fica VERB
ai PROPN
veno ADJ
vc PROPN
conseguiu VERB
goixtosaaa NOUN
alguem VERB
sabe NOUN
valentao VERB
preso VERB
economiza VERB
xandao NOUN
tora ADJ
bambu NOUN
antes ADV
c

newprofilepicnatualizando VERB
fotinha NOUN
excelente ADJ
agressivo ADJ
ajude NOUN
inconsciente ADV
safado VERB
ntsc NOUN
supremo PROPN
tribunal PROPN
federal ADJ
suprema PROPN
corte NOUN
brasiliabrazil PROPN
supremo PROPN
tribunal PROPN
federal ADJ
stf VERB
quebra NOUN
facilidade NOUN
vida ADJ
soubesse ADJ
unha PROPN
redonda VERB
vida NOUN
feito VERB
antes ADV
the PROPN
rock PROPN
parece VERB
geladeira NOUN
porque SCONJ
assim ADV
😪 NUM
🤡 NOUN
assistindo VERB
black VERB
adam VERB
hojea NOUN
fa ADJ
heroi PROPN
fala VERB
adotou VERB
🥺 NOUN
acharem VERB
ruim ADJ
ainda ADV
manda VERB
lavine NOUN
junto ADV
arredondei VERB
ficou VERB
chique ADJ
lixo NOUN
humano ADJ
caralho ADJ
🥺 NOUN
q CCONJ
vc PROPN
provavelmente ADV
deve VERB
gravar VERB
mandar VERB
email NOUN
... PUNCT
email NOUN
vou AUX
falar VERB
misteriosamente ADV
kkkkkk VERB
todo DET
trabalho NOUN
qrida VERB
😍 ADP
😍 NOUN
vi VERB
achei VERB
delirando VERB
agora ADV
acho VERB
coletivo ADJ
cabecao VERB
safado VERB
fala VERB
bicha NOUN
😂

🕗 PROPN
dong VERB
dong VERB
dong PROPN
dong PROPN
dong PROPN
dong PROPN
dong PROPN
dong VERB
jogo NOUN
apostava VERB
flu PROPN
consegue VERB
fazer VERB
parte NOUN
perdendo VERB
ritmo NOUN
visto VERB
pergunta VERB
quer VERB
calar VERB
andersontorres NOUN
aparece VERB
escoltado VERB
policia VERB
miami NOUN
convidado VERB
deixar VERB
pais NOUN
protecao NOUN
expulsao ADJ
coincidencia NOUN
ciro PROPN
nogueira VERB
mourao PROPN
embarcaram VERB
agora ADV
miami VERB
certa DET
praias NOUN
simmmmm NOUN
espero VERB
galera ADJ
desfrutado VERB
porque SCONJ
nada PRON
esconder VERB
😏 PROPN
nasci NOUN
ontem ADV
pode VERB
perdao NOUN
desabafei ADJ
c CCONJ
mistki VERB
lancar VERB
pearl NOUN
aguento VERB
apagar VERB
coisa NOUN
desse ADP
celular VERB
ainda ADV
fica VERB
dizendo VERB
q CCONJ
armazenamento NOUN
cheio ADJ
vagbuna NOUN
deveria VERB
ter VERB
vergonha NOUN
destrinche VERB
fez VERB
time NOUN
mulher ADJ
njnta VERB
sebosa NOUN
veeeei NOUN
vitoria ADJ
vontade ADJ
barueri ADJ
fluminense ADJ
esperava

cansado VERB
alem VERB
sim ADV
🇧 PROPN
🇷 NOUN
🇧 PROPN
🇷 PROPN
🇧 PROPN
🇷 VERB
sensacionallllnela NOUN
filme NOUN
deixa VERB
depre ADV
todo DET
mundo NOUN
acha VERB
nada ADV
comer VERB
rezar VERB
amar VERB
falta NOUN
pedra NOUN
forja VERB
sombria NOUN
pra SCONJ
upar VERB
cao NOUN
caca VERB
pra ADP
continuo VERB
receber VERB
varios NOUN
cadastros ADJ
exato NOUN
galaseca ADJ
🤣 NOUN
🤣 PROPN
🤣 PROPN
🤣 PROPN
brasil PROPN
assim ADV
diversidade NOUN
girias ADJ
ai PROPN
morri PROPN
parabens NOUN
deputado ADJ
populacao PROPN
precisa VERB
justica NOUN
social ADJ
contemplando VERB
trara NOUN
bom ADJ
️ NOUN
momento NOUN
mulherzinha VERB
picanha VERB
veio VERB
pao ADP
diabo NOUN
amassou VERB
servido VERB
ver VERB
crianca NOUN
trabalhando VERB
parte NOUN
coracao ADJ
dei VERB
pro SCONJ
menorzin NOUN
pedir VERB
nada PRON
troca NOUN
pedi VERB
pra ADP
casa NOUN
sorriso ADJ
deu VERB
gratificante NOUN
ganhei ADJ
dia NOUN
maybe NOUN
one NOUN
day ADP
ill PROPN
be PROPN
an PROPN
honest PROPN
mannup NOUN
til PR

🕖 PROPN
dong VERB
dong PROPN
dong PROPN
dong PROPN
dong PROPN
dong PROPN
dong VERB
general ADJ
braga PROPN
neto PROPN
choro VERB
pode VERB
durar VERB
noite NOUN
alegria ADJ
vem VERB
manhantudo NOUN
passa VERB
nespero ADJ
logo ADJ
desanime NOUN
usava VERB
demais ADV
imprimir VERB
provas NOUN
trabalhos NOUN
matematica ADJ
ai PROPN
ai PROPN
q PROPN
odio NOUN
cheguei VERB
conclusao NOUN
mae PROPN
ama VERB
bolsonaro ADJ
porque SCONJ
autoritario ADJ
machista ADJ
grotesco ADJ
quanto ADP
pai NOUN
ai ADP
tudo PRON
explica VERB
novo ADJ
secretario NOUN
interino ADJ
mulher ADJ
igualdade NOUN
racial ADJ
parana ADJ
aka PROPN
russia PROPN
brasileira ADJ
hein ADV
rogerio ADJ
carboni PROPN
nomeado VERB
governador NOUN
ratinho ADJ
jr ADP
militares NOUN
tentaram VERB
fazer VERB
acabaram VERB
criando VERB
millor NOUN
jaguar VERB
jo ADP
soares PROPN
chico ADJ
anysio PROPN
henfil VERB
ziraldo X
pleiade NOUN
humoristas ADJ
expunham VERB
corrupcao ADJ
ridiculo PROPN
governo PROPN
lula ADJ
pode VERB
tentar VE

...

homens NOUN
disponiveis ADJ
certeza NOUN
pagos VERB
pra ADP
isson"missao NOUN
dada VERB
missao NOUN
cumprida"nna NOUN
esquerda ADJ
inteligente ADJ
lucraram VERB
tiro NOUN
chapeuneles ADJ
armaram VERB
baderna ADJ
certeza VERB
nmovimento NOUN
elamente ADV
divulgadonsairam VERB
salvadores NOUN
patria ADJ
saonestao VERB
unidos NOUN
todos DET
poderes NOUN
ah VERB
pronto ADJ
agora ADV
viciado VERB
musica ADJ
pega VERB
guanabara NOUN
vem VERB
morte NOUN
horrivel ADJ
😂 X
presos VERB
uai VERB
kkkkkk NOUN
queria VERB
churrasco NOUN
picanha VERB
heineken ADJ
gentes NOUN
vc ADJ
seguem VERB
signos ADJ
vcs NOUN
risca ADJ
acham VERB
boa ADJ
referencia NOUN
pra ADP
tomada VERB
decisoes ADJ
vida PROPN
vcs PROPN
amorde ADJ
pode VERB
esperar VERB
sentado VERB
rainha VERB
louca ADJ
cada DET
dia NOUN
sedenta ADJ
cortemlhe PRON
cabeca PROPN
🥚 PROPN
🦑 NOUN
nem ADV
dorme ADJ
pensando VERB
proxima DET
ordem NOUN
dara VERB
ainda ADV
aguardando VERB
fila NOUN
pra SCONJ
pegar VERB
hehe VERB
acabou VERB
publicar V

ninguem VERB
negocia VERB
terroristas ADJ
prendenjanones NOUN
autorizo VERB
abandou VERB
carinho NOUN
😩 PUNCT
😩 PROPN
😩 PROPN
😩 PROPN
😩 PROPN
😩 PROPN
😩 PROPN
😩 PROPN
😩 PROPN
😩 PROPN
😩 PROPN
😩 PROPN
😩 PROPN
nada🫶🏻 NOUN
sabia VERB
fazer VERB
aprender VERB
pra SCONJ
mandar VERB
aqui ADV
kkkk ADJ
inesquecivel ADJ
historia NOUN
ainda ADV
bem ADV
amigos NOUN
fofoqueiros ADJ
traziam VERB
informacoes VERB
kkkkkkkk ADJ
ansiedade NOUN
aqui ADV
deixa VERB
mentirosa ADJ
familia VERB
educacao PROPN
deu VERB
estupido VERB
demais ADV
dino PROPN
avisou VERB
excesso NOUN
ignorancia VERB
mesma ADJ
logica NOUN
certa DET
religiao VERB
prefere VERB
deixar VERB
parente ADJ
morrer VERB
ter AUX
submeter VERB
transfusao NOUN
sangue NOUN
exagero ADJ
worlds VERB
new PROPN
largest PROPN
wind PROPN
turbine VERB
sweeps VERB
football PROPN
fields PROPN
per PROPN
spin VERB
energy VERB
cleanenergy VERB
sustainability ADJ
gosto VERB
terror ADJ
vejo PROPN
amem VERB
🙏 PROPN
brinca PROPN
nome PROPN
deus VERB
apenas ADV
ve

cadeia VERB
verdadeiros ADJ
criminosos NOUN
alexandre NOUN
primeiro ADV
medo NOUN
sei VERB
primeiro ADJ
nome NOUN
😢 ADJ
porque SCONJ
cabe VERB
acabei VERB
ver VERB
album VERB
formatura NOUN
nno ADP
sobrou VERB
beleza NOUN
faltou VERB
grana ADJ
🥺 PRON
😭 VERB
nuvem VERB
negra ADJ
cima ADP
pior ADJ
hora NOUN
nomeacao NOUN
secretario NOUN
justica ADJ
andersonn NOUN
inacreditavel ADJ
mulher NOUN
defensora ADJ
publica VERB
comemorar VERB
vandalismo NOUN
minimo ADJ
devia VERB
suspensa VERB
atribuicoes NOUN
inacreditavel ADJ
q PRON
medica VERB
supostamente ADV
esclarecida VERB
apoie NOUN
atos VERB
terroristas ADJ
cadeia VERB
nela PRON
doses NOUN
hidrocliroquina ADJ
diarian PROPN
esports NOUN
esporte ADJ
quanto ADV
pablo VERB
vittar VERB
mulher PROPN
esports NOUN
parabens NOUN
querida VERB
regiao NOUN
administrativa ADJ
lago PROPN
norte ADJ
distrito ADJ
federal ADJ
hoje ADV
completa VERB
anos NOUN
feliz ADJ
morar VERB
aqui ADV
lago PRON
norte ADJ
desde ADP
longinquo ADJ
ano NOUN
deveria VERB
re

esquerda NOUN
inteligente ADJ
lucraram VERB
tiro NOUN
chapeuneles ADJ
armaram VERB
baderna ADJ
certeza VERB
nmovimento NOUN
elamente ADV
divulgadonsairam VERB
salvadores NOUN
patria ADJ
saonestao VERB
unidos NOUN
todos DET
poderes NOUN
imprensa ADJ
podre ADJ
saudades NOUN
existiam VERB
blusas ADJ
tamanho NOUN
normal ADJ
lojas NOUN
encurralaramnos NOUN
benjamin PROPN
gostava VERB
prova NOUN
entao ADJ
ficava VERB
super ADJ
animada VERB
ehehhe ADJ
ninfelizmente ADV
coisas NOUN
mudaram VERB
kkkk ADJ
🫠 VERB
alo NOUN
tal ADV
tomar VERB
iniciativa NOUN
identica ADJ
... PUNCT
vcs NOUN
fazer VERB
msm NOUN
ne PUNCT
vai AUX
responsabilizada ADJ
nada ADV
clic PROPN
fala VERB
si PRON
julius ADV
quero VERB
saber VERB
disso PRON
deixa VERB
quieta NOUN
lembrando VERB
concurso NOUN
area ADJ
seguranca NOUN
rigorosa ADJ
investigacao NOUN
vida ADJ
pregressanparticipacao PROPN
ato ADJ
terrorista ADJ
deve VERB
pegar VERB
bem ADV
antecedente ADV
bbb ADP
c NOUN
q PRON
vai AUX
baaaaum PROPN
nq PROPN
gato NOUN


KeyboardInterrupt: 

In [19]:
nlp = spacy.load("pt_core_news_sm")
doc = nlp()
print(doc.text)
for token in doc:
    print(token.text, token.pos_)

Apple tá querendo comprar uma startup por 100 milhoes de dolares
Apple PROPN nsubj
tá AUX aux
querendo VERB ROOT
comprar VERB xcomp
uma DET det
startup NOUN obj
por ADP case
100 NUM nummod
milhoes NOUN obl
de ADP case
dolares NOUN nmod
